In [1]:
import org.apache.spark.sql.SQLContext
import org.apache.spark.sql.types.{StructType, StructField, StringType, DoubleType};
import com.databricks.spark.xml._

val sqlContext = new SQLContext(sc)
import sqlContext.implicits._
sqlContext.sql("set spark.sql.caseSensitive=true") 

Waiting for a Spark session to start...

Name: org.apache.spark.SparkException
Message: Application application_1572457089359_0028 failed 2 times due to AM Container for appattempt_1572457089359_0028_000002 exited with  exitCode: -1000
Failing this attempt.Diagnostics: [2019-11-04 16:41:22.498]File file:/N/u/yan30/Carbonate/.sparkStaging/application_1572457089359_0028/datanucleus-core-4.1.17.jar does not exist
java.io.FileNotFoundException: File file:/N/u/yan30/Carbonate/.sparkStaging/application_1572457089359_0028/datanucleus-core-4.1.17.jar does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:641)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:867)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:631)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:442)
	at org.apache.hadoop.yarn.util.FSDownload.verifyAndCopy(FSDownload.java:269)
	at org.apache.hadoop.yarn.util.F

In [14]:
import java.io.File
import scala.collection.JavaConversions._ // important for 'foreach'
import org.apache.commons.io.FileUtils

FileUtils.listFiles(new File("/N/dc2/projects/IUNI_MSAcademic/wos_raw/2017_old/"), Array("foo"), true).foreach{ f =>

}

lastException: Throwable = null


In [2]:
import org.apache.hadoop.fs.{FileSystem, FileUtil, Path}

val fs = FileSystem.get(sc.hadoopConfiguration)
val status = fs.listStatus(new Path("/data1"))
status.foreach(x=> println(x.getPath))

Waiting for a Spark session to start...

hdfs://iuni2:8020/data1/parquet


fs = DFS[DFSClient[clientName=DFSClient_NONMAPREDUCE_383504919_58, ugi=hdfs (auth:SIMPLE)]]
status = Array(HdfsLocatedFileStatus{path=hdfs://iuni2:8020/data1/parquet; isDirectory=true; modification_time=1572555443195; access_time=0; owner=hdfs; group=supergroup; permission=rwxr-xr-x; isSymlink=false; hasAcl=false; isEncrypted=false; isErasureCoded=false})


Array(HdfsLocatedFileStatus{path=hdfs://iuni2:8020/data1/parquet; isDirectory=true; modification_time=1572555443195; access_time=0; owner=hdfs; group=supergroup; permission=rwxr-xr-x; isSymlink=false; hasAcl=false; isEncrypted=false; isErasureCoded=false})

In [20]:
//val sqlContext = new SQLContext(sc)
val customSchema = StructType(Array(
  StructField("UID", StringType, nullable = true),
  StructField("static_data", StructType(Array( StructField("fullrecord_metadata", StructType(Array(
        //StructField("titles", StructType(Array(StructField("title", StringType, nullable = false)))),
        //StructField("names", StructType(Array(StructField("name", StringType, nullable = false)))),
        StructField("references", StructType(Array(StructField("reference", StructType(Array(StructField("UID", StringType, nullable = false)))))))
  ))))))))
val df = sqlContext.read
  .format("com.databricks.spark.xml")
  .option("rowTag", "REC")
  //.schema(customSchema)
  .load("file:///N/dc2/projects/IUNI_MSAcademic/wos_raw/2017_old/WR_2015_20160330230947_CORE_0001.xml.gz")

df.show(2)
df.printSchema()

lastException = null


Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 0 in stage 2.0 failed 4 times, most recent failure: Lost task 0.3 in stage 2.0 (TID 60, iuni4, executor 7): java.io.FileNotFoundException: File file:/N/dc2/projects/IUNI_MSAcademic/wos_raw/2017_old/WR_2015_20160330230947_CORE_0001.xml.gz does not exist
	at org.apache.hadoop.fs.RawLocalFileSystem.deprecatedGetFileStatus(RawLocalFileSystem.java:641)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileLinkStatusInternal(RawLocalFileSystem.java:867)
	at org.apache.hadoop.fs.RawLocalFileSystem.getFileStatus(RawLocalFileSystem.java:631)
	at org.apache.hadoop.fs.FilterFileSystem.getFileStatus(FilterFileSystem.java:442)
	at org.apache.hadoop.fs.ChecksumFileSystem$ChecksumFSInputChecker.<init>(ChecksumFileSystem.java:146)
	at org.apache.hadoop.fs.ChecksumFileSystem.open(ChecksumFileSystem.java:347)
	at org.apache.hadoop.fs.FileSystem.open(FileSystem.java:898)
	at com.databricks.spark.xml.XmlRecordReader.initial

In [17]:
import org.apache.spark.sql.SaveMode
df.repartition(1000).write.mode(SaveMode.Overwrite).parquet("file:////N/dc2/projects/IUNI_MSAcademic/wos_raw/2017_old/parquet/")

Name: Unknown Error
Message: lastException: Throwable = null
<console>:46: error: not found: value df
       df.repartition(1000).write.mode(SaveMode.Overwrite).parquet("file:////N/dc2/projects/IUNI_MSAcademic/wos_raw/2017_old/parquet/")
       ^

StackTrace: 

In [2]:
val WoS = spark.read.format("parquet").load("/data1/parquet/")
val WoS2 = WoS.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.abstracts.abstract.abstract_text.p","static_data.summary.doctypes.doctype",
                       "static_data.summary.pub_info._pubyear","static_data.summary.names.name.display_name")
WoS2.printSchema

root
 |-- UID: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- _value: string (nullable = true)
 |-- p: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- doctype: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- _pubyear: long (nullable = true)
 |-- display_name: array (nullable = true)
 |    |-- element: string (containsNull = true)



WoS = [UID: string, _r_id_disclaimer: string ... 13 more fields]
WoS2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 4 more fields]


[UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 4 more fields]

In [3]:
WoS2.describe().filter($"summary" === "count").show

java.security.AccessControlException: access denied org.apache.derby.security.SystemPermission( "engine", "usederbyinternals" )
	at java.security.AccessControlContext.checkPermission(AccessControlContext.java:472)
	at java.security.AccessController.checkPermission(AccessController.java:884)
	at org.apache.derby.iapi.security.SecurityUtil.checkDerbyInternalsPrivilege(Unknown Source)
	at org.apache.derby.iapi.services.monitor.Monitor.startMonitor(Unknown Source)
	at org.apache.derby.iapi.jdbc.JDBCBoot$1.run(Unknown Source)
	at java.security.AccessController.doPrivileged(Native Method)
	at org.apache.derby.iapi.jdbc.JDBCBoot.boot(Unknown Source)
	at org.apache.derby.iapi.jdbc.JDBCBoot.boot(Unknown Source)
	at org.apache.derby.jdbc.EmbeddedDriver.boot(Unknown Source)
	at org.apache.derby.jdbc.EmbeddedDriver.<clinit>(Unknown Source)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImp

In [3]:
val yearCount = WoS2.groupBy("_pubyear").count()
yearCount.show(yearCount.count.toInt, false)

java.security.AccessControlException: access denied org.apache.derby.security.SystemPermission( "engine", "usederbyinternals" )
	at java.security.AccessControlContext.checkPermission(AccessControlContext.java:472)
	at java.security.AccessController.checkPermission(AccessController.java:884)
	at org.apache.derby.iapi.security.SecurityUtil.checkDerbyInternalsPrivilege(Unknown Source)
	at org.apache.derby.iapi.services.monitor.Monitor.startMonitor(Unknown Source)
	at org.apache.derby.iapi.jdbc.JDBCBoot$1.run(Unknown Source)
	at java.security.AccessController.doPrivileged(Native Method)
	at org.apache.derby.iapi.jdbc.JDBCBoot.boot(Unknown Source)
	at org.apache.derby.iapi.jdbc.JDBCBoot.boot(Unknown Source)
	at org.apache.derby.jdbc.EmbeddedDriver.boot(Unknown Source)
	at org.apache.derby.jdbc.EmbeddedDriver.<clinit>(Unknown Source)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance0(Native Method)
	at sun.reflect.NativeConstructorAccessorImpl.newInstance(NativeConstructorAccessorImp

Name: java.lang.UnsupportedOperationException
Message: empty.max
StackTrace:   at scala.collection.TraversableOnce$class.max(TraversableOnce.scala:229)
  at scala.collection.AbstractTraversable.max(Traversable.scala:104)
  at org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator$.updateAndGetCompilationStats(CodeGenerator.scala:1354)
  at org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator$.org$apache$spark$sql$catalyst$expressions$codegen$CodeGenerator$$doCompile(CodeGenerator.scala:1291)
  at org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator$$anon$1.load(CodeGenerator.scala:1372)
  at org.apache.spark.sql.catalyst.expressions.codegen.CodeGenerator$$anon$1.load(CodeGenerator.scala:1369)
  at org.spark_project.guava.cache.LocalCache$LoadingValueReference.loadFuture(LocalCache.java:3599)
  at org.spark_project.guava.cache.LocalCache$Segment.loadSync(LocalCache.java:2379)
  at org.spark_project.guava.cache.LocalCache$Segment.lockedGetOrLoad(LocalCache.

In [12]:
WoS2.describe().filter($"summary" === "count").show

Name: org.apache.spark.SparkException
Message: Job aborted due to stage failure: Task 6 in stage 12.0 failed 4 times, most recent failure: Lost task 6.3 in stage 12.0 (TID 1032, iuni9, executor 73): java.lang.RuntimeException: Found duplicate field(s) "_VALUE": [_VALUE, _value] in case-insensitive mode
	at org.apache.spark.sql.execution.datasources.parquet.ParquetReadSupport$$anonfun$clipParquetGroupFields$2$$anonfun$apply$3.apply(ParquetReadSupport.scala:315)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetReadSupport$$anonfun$clipParquetGroupFields$2$$anonfun$apply$3.apply(ParquetReadSupport.scala:311)
	at scala.Option.map(Option.scala:146)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetReadSupport$$anonfun$clipParquetGroupFields$2.apply(ParquetReadSupport.scala:311)
	at org.apache.spark.sql.execution.datasources.parquet.ParquetReadSupport$$anonfun$clipParquetGroupFields$2.apply(ParquetReadSupport.scala:308)
	at scala.collection.TraversableLike$$anonfun$m

In [32]:
val esci2 = esci.select("UID","dynamic_data.cluster_related.identifiers.identifier","static_data.abstracts.abstract.abstract_text.p","static_data.summary.doctypes.doctype",
                       "static_data.summary.pub_info._pubyear","static_data.summary.names.name.display_name")
esci2.printSchema

root
 |-- UID: string (nullable = true)
 |-- identifier: array (nullable = true)
 |    |-- element: struct (containsNull = true)
 |    |    |-- _VALUE: string (nullable = true)
 |    |    |-- _type: string (nullable = true)
 |    |    |-- _value: string (nullable = true)
 |-- p: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- doctype: array (nullable = true)
 |    |-- element: string (containsNull = true)
 |-- _pubyear: long (nullable = true)
 |-- display_name: array (nullable = true)
 |    |-- element: string (containsNull = true)



esci2 = [UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 4 more fields]


[UID: string, identifier: array<struct<_VALUE:string,_type:string,_value:string>> ... 4 more fields]

In [34]:
val esciCount = esci2.groupBy("_pubyear").count()
esciCount.show(yearCount.count.toInt, false)

[Stage 81:=================================================>        (6 + 1) / 7]+--------+------+
|_pubyear|count |
+--------+------+
|2016    |291622|
|2018    |25    |
|2017    |307040|
+--------+------+



esciCount = [_pubyear: bigint, count: bigint]


[_pubyear: bigint, count: bigint]

In [3]:
spark.sparkContext.uiWebUrl

Some(http://iuni2:4040)